In [1]:
import pandas as pd
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

In [11]:
DAYSBACK = 300
PARMS = """('SAIL23_H2_0p50to1p05_PerfectYield','SAIL23_H2_0p65to1p05_PerfectYield')"""
FAMILY_CODE = """('Q6')"""

In [20]:
QUERY = f"""
SELECT
  lot_Id, wafer_Id, normalized_TestX as chipX, normalized_TestY as chipY,
  family_Code, Last_test_date AS date, parm_Label as parameter,
  parmYield as yield, unitcell_X as kerf_X, unitcell_Y as kerf_Y
FROM DMIW.ChipParmFactR cpfr
  JOIN DMIW_SYSPRC.TestParm tp ON tp.TestParmKey = cpfr.TestParmKey
  JOIN DMIW_SYSPRC.Geography g ON g.geographyKey = cpfr.geographyKey
  JOIN DMIW_SYSPRC.TestedWafer tw ON cpfr.testedWaferKey = tw.testedWaferKey
WHERE
  Last_test_date >= (current date - {DAYSBACK} days)
  AND Tech_id = '7HPP' AND Level = 'H2'
  AND cpfr.parmValue is NOT NULL AND ABS(parmValue) < 1e25 
  AND tp.parm_Label IN {PARMS}
  AND family_Code IN {FAMILY_CODE}
  
ORDER BY
  lot_Id, wafer_Id, normalized_TestX, normalized_TestY, parm_Label, level, tw.calcdefs
"""

In [22]:
ibmdata.isdw.query(QUERY)

,lot_id,wafer_id,chipx,chipy,family_code,date,calcdefs,step,parameter,yield,kerf_x,kerf_y
0,A1AVP.1,A1AVP-04,9,48,Q6,2021-05-07,SSL21,SSL2,SAIL23_H2_0p50to1p05_PerfectYield,100.00,2,5
1,A1AVP.1,A1AVP-04,9,48,Q6,2021-05-07,SSL21,SSL2,SAIL23_H2_0p65to1p05_PerfectYield,100.00,2,5
2,A1AVP.1,A1AVP-04,9,59,Q6,2021-05-07,SSL21,SSL2,SAIL23_H2_0p50to1p05_PerfectYield,100.00,2,6
3,A1AVP.1,A1AVP-04,9,59,Q6,2021-05-07,SSL21,SSL2,SAIL23_H2_0p65to1p05_PerfectYield,100.00,2,6
4,A1AVP.1,A1AVP-04,9,70,Q6,2021-05-07,SSL21,SSL2,SAIL23_H2_0p50to1p05_PerfectYield,100.00,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...
310357,AZBAU.1,AZBAU-25,99,86,XQ,2021-07-25,SSL21,SSL2,SAIL23_H2_0p65to1p05_PerfectYield,100.00,12,7
310358,AZBAU.1,AZBAU-25,99,101,XQ,2021-07-25,SSL21,SSL2,SAIL23_H2_0p50to1p05_PerfectYield,100.00,12,8
310359,AZBAU.1,AZBAU-25,99,101,XQ,2021-07-25,SSL21,SSL2,SAIL23_H2_0p65to1p05_PerfectYield,100.00,12,8
310360,AZBAU.1,AZBAU-25,99,116,XQ,2021-07-25,SSL21,SSL2,SAIL23_H2_0p50to1p05_PerfectYield,100.00,12,9


# Metrology

In [29]:
QUERY = f"""
SELECT
  lot_Id, wafer_Id, normalized_TestX as chipX, normalized_TestY as chipY, 
  family_Code, measurement_date, meas_Pd_Id as step, dcItem_Name as parameter,
  dcItem_Value as measurement, unitcell_X as kerf_X, unitcell_Y as kerf_Y
FROM DMIW.MeasuredRawFact mrf
  JOIN DMIW_SYSPRC.MeasuredParm mp ON mp.MeasuredParmKey = mrf.MeasuredParmKey 
  JOIN DMIW_SYSPRC.Geography g ON g.GeographyKey = mrf.GeographyKey 
  JOIN DMIW_SYSPRC.MeasuredWafer mw ON mw.MeasuredWaferKey = mrf.MeasuredWaferKey 
WHERE
  Measurement_date >= (current date - {DAYSBACK} days) 
  AND Tech_id = '7HPP' 
  AND Family_code = 'Q6'
  AND Meas_pd_id like '%KU072615'
  AND meas_Pd_Id <> '' AND dcItem_Value IS NOT NULL
  AND UCASE(dcItem_Name) NOT IN ('XCOORD', 'YCOORD', 'XSITE', 'YSITE', 'FIELD X', 'FIELD Y', 'SITE X', 'SITE Y', 'SITEID', 'ROW', 'COLUMN') 
  AND (ucase(rtrim(meas_Pd_Id) || ' ' || dcItem_Name) like '%PC%OCD%' )  
ORDER BY 
  lot_Id, wafer_Id,normalized_TestX, normalized_TestY, dcItem_Name, proc_Level
"""

In [30]:
ibmdata.isdw.query(QUERY)

,lot_id,wafer_id,chipx,chipy,family_code,measurement_date,step,parameter,measurement,kerf_x,kerf_y
0,A1AVP.1,A1AVP-01,9,70,Q6,2021-02-10,KU072615,PC OCD (Logic 60CPP),15.802353,2,7
1,A1AVP.1,A1AVP-01,9,70,Q6,2021-02-10,KU072615,PC OCD (SRAM 54CPP),14.715049,2,7
2,A1AVP.1,A1AVP-01,27,48,Q6,2021-02-10,KU072615,PC OCD (Logic 60CPP),15.758132,4,5
3,A1AVP.1,A1AVP-01,27,48,Q6,2021-02-10,KU072615,PC OCD (SRAM 54CPP),14.995612,4,5
4,A1AVP.1,A1AVP-01,27,92,Q6,2021-02-10,KU072615,PC OCD (Logic 60CPP),15.759784,4,9
...,...,...,...,...,...,...,...,...,...,...,...
18663,AZAN4.1,AZAN4-23,81,70,Q6,2021-02-27,KU072615,PC OCD (SRAM 54CPP),15.102416,10,7
18664,AZAN4.1,AZAN4-23,81,92,Q6,2021-02-27,KU072615,PC OCD (Logic 60CPP),15.749623,10,9
18665,AZAN4.1,AZAN4-23,81,92,Q6,2021-02-27,KU072615,PC OCD (SRAM 54CPP),14.998904,10,9
18666,AZAN4.1,AZAN4-23,99,59,Q6,2021-02-27,KU072615,PC OCD (Logic 60CPP),15.893147,12,6
